# Άσκηση: Σύγκριση του Gausian Naive Bayes και του kNN στo Pima Indians Diabetes dataset
![1889 Photograph shows half-length portrait of two Pima Indians, facing front, wearing bead necklaces.](https://i.pinimg.com/236x/60/05/76/600576905d4ad5bb1a9c3e3387b397ca--pima-indians-native-american-indians.jpg "1889 Photograph shows half-length portrait of two Pima Indians, facing front, wearing bead necklaces.")

Διαβάστε το ["pima-indians-diabetes.data.csv"](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv) σε ένα numpy array data και ξεχωρίστε features και labels.

Για 40% test set: 
1. υπολογίστε την πρόβλεψη του Gaussian Naive Bayes με τη μέθοδο predict(). 
2. πάρτε τις προβλέψεις ενός kNN με k=5
3. Για έναν ταξινομητή kNN, με 3-fold cross validation και με μετρική 'f1_weighted' υπολογίστε το βέλτιστο k στο train set (maximum k=50). 
4. εκτυπώστε με την "classification_report" τα precision, recall, f1, support για τον NB, τον kNN με k=5 και με το k που έχει προκύψει από cross validation.
5. Κάντε 3 runs και αποθηκεύστε σε ένα κελί markdown το average F1 του non optimized και του optimized kNN. Πόσο % έχει βελτιβωθεί η επίδοσή του;

hint: Για τη δημιουργία πινάκων σε markdown μπορείτε να χρησιμοποιείτε ένα [markdown table generator](https://www.tablesgenerator.com/markdown_tables)

In [5]:
!pip install --upgrade sklearn
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install --upgrade scipy

Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.23.4)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.15.4)
Requirement already up-to-date: scipy in /usr/local/lib/python3.6/dist-packages (1.1.0)


In [0]:
CSV_filename = "pima-indians-diabetes.data.csv"

In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving pima-indians-diabetes.data.csv to pima-indians-diabetes.data.csv
User uploaded file "pima-indians-diabetes.data.csv" with length 23278 bytes


In [8]:
import pandas as pd

df = pd.read_csv(CSV_filename, header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [0]:
import numpy as np
np_data = df.values
features = np_data[:,0:-1]
labels = np_data[:,-1].astype(int)

In [10]:
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.30)
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
model = gnb.fit(train, train_labels)
gnb_pred = gnb.predict(test)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train, train_labels)
knn_pred = knn.predict(test)


from sklearn.model_selection import cross_val_score
myList = list(range(1,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))
cv_scores = []
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, train, train_labels, cv=3, scoring='f1_weighted')
    cv_scores.append(scores.mean())
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("\noptimal number of k in the train set by f1 (weighted), 10-folds (30% test set):", optimal_k)

knn = KNeighborsClassifier(n_neighbors=optimal_k)
knn.fit(train, train_labels)
knn_pred1 = knn.predict(test)

print("\nClassification Precision, Recall, F1 on the Pima Indian Diabetes Dataset (30% test set)")
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
print("\nGaussian Naive Bayes\n")
print(classification_report(test_labels, gnb_pred))
print("\nk Nearest Neighbours k = 5\n")
print(classification_report(test_labels, knn_pred))
print("\nk Nearest Neighbours k optimized (%s)" % optimal_k, "\n")
print(classification_report(test_labels, knn_pred1))


optimal number of k in the train set by f1 (weighted), 10-folds (30% test set): 11

Classification Precision, Recall, F1 on the Pima Indian Diabetes Dataset (30% test set)

Gaussian Naive Bayes

             precision    recall  f1-score   support

          0       0.81      0.88      0.84       154
          1       0.70      0.58      0.64        77

avg / total       0.77      0.78      0.77       231


k Nearest Neighbours k = 5

             precision    recall  f1-score   support

          0       0.77      0.81      0.79       154
          1       0.57      0.52      0.54        77

avg / total       0.70      0.71      0.71       231


k Nearest Neighbours k optimized (11) 

             precision    recall  f1-score   support

          0       0.78      0.88      0.83       154
          1       0.68      0.51      0.58        77

avg / total       0.75      0.76      0.75       231



| Run | k=5   | k optimized |
|-----|-------|-------------|
| 1   | 0.69  | 0.74        |
| 2   | 0.69  | 0.67       |
| 3   | 0.71  | 0.73        |
| AVG | 0.697 | 0.713        |

0.697 (1+βελτίωση) = 0.713 $\Rightarrow$
βελτίωση = 0.713/0.697 - 1  $\Rightarrow$
βελτίωση = 0,023 δηλαδή 2,3%